In [2]:
import string
import nltk
from nltk import word_tokenize
nltk.download('stopwords')
import numpy as np
from nltk.corpus import stopwords
from nltk.corpus import words
from nltk.stem import WordNetLemmatizer
from nltk import tokenize
from nltk.tag import pos_tag
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [1]:
from google.colab import drive

In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
with open('/content/drive/My Drive/MyDive/POS_items/test.words', 'r') as f:
  listing = f.readlines()
listing

FileNotFoundError: ignored

In [3]:
def preprocessing(str):
    #stopword removal
    Stop = stopwords.words('english')
    #remove punctuation
    no_punc = [char for char in str if char not in string.punctuation]
    no_punc = ''.join(no_punc)
    #add word lemmatization.
    
    #return ' '.join([word for word in no_punc.split() if (word.lower() not in Stop)])
    return ' '.join([word for word in no_punc.split()])

In [10]:
def pos_tagging(s):
    nopunc_sentence = preprocessing(s)
    print(nopunc_sentence)
    lemmatizer = WordNetLemmatizer()
    tags = pos_tag(nltk.word_tokenize(nopunc_sentence))
    dict_tag = {'NN':[], 'VB':[], 'OTH':[]} #only three categories at the moment, can add all categories coming from pos_tag programmatically.
    for tag in tags:
        if tag[1] in ["NN","NNS","NNP","NNPS"]: #NN-singular noun, NNS- noun plural, NNP-proper noun singular, NNPS- proper noun plural.
            dict_tag['NN'].append(tag[0])
        elif tag[1] in ["VB","VBD","VBG","VBN","VBP","VBZ"]: #VB: verb in base form, VBG: verb in present participle, VBN: past participle taken, VBZ: verb singular, present, non-3rd person take, VBP:verb, singular present
            dict_tag['VB'].append(tag[0])
        else:
            dict_tag['OTH'].append(tag[0])
    return dict_tag

In [11]:
string2 = "You should be ashamed of yourself! for slandering Newcastle United"
tags2 = pos_tagging(string2)
print(tags2)

You should be ashamed of yourself for slandering Newcastle United
{'NN': ['Newcastle', 'United'], 'VB': ['be', 'ashamed', 'slandering'], 'OTH': ['You', 'should', 'of', 'yourself', 'for']}


In [12]:
print(preprocessing(string2))

You should be ashamed of yourself for slandering Newcastle United


In [13]:
lemmatizer = WordNetLemmatizer()

list2= nltk.word_tokenize(preprocessing(string2))
lemmatized = ' '.join([lemmatizer.lemmatize(w) for w in list2])
print(list2)
print(lemmatized)
print(pos_tagging(lemmatized))
print(pos_tagging(preprocessing(string2)))
print(pos_tagging(string2))

['You', 'should', 'be', 'ashamed', 'of', 'yourself', 'for', 'slandering', 'Newcastle', 'United']
You should be ashamed of yourself for slandering Newcastle United
You should be ashamed of yourself for slandering Newcastle United
{'NN': ['Newcastle', 'United'], 'VB': ['be', 'ashamed', 'slandering'], 'OTH': ['You', 'should', 'of', 'yourself', 'for']}
You should be ashamed of yourself for slandering Newcastle United
{'NN': ['Newcastle', 'United'], 'VB': ['be', 'ashamed', 'slandering'], 'OTH': ['You', 'should', 'of', 'yourself', 'for']}
You should be ashamed of yourself for slandering Newcastle United
{'NN': ['Newcastle', 'United'], 'VB': ['be', 'ashamed', 'slandering'], 'OTH': ['You', 'should', 'of', 'yourself', 'for']}


In [14]:
m = np.matrix('0.2 0.2 0.6;0.4 0.3 0.3;0.2 0.3 0.5')
print(m)

[[0.2 0.2 0.6]
 [0.4 0.3 0.3]
 [0.2 0.3 0.5]]


Generating transition and emission matrices -> tarini_HMM


In [15]:
import numpy as np
import math
import string
from collections import defaultdict
import pandas as pd

In [16]:
#Initialization

setPunctuation = set(string.punctuation)
suffixNoun = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er", "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness", "or", "ry", "scape", "ship", "ty"]
suffixVerb = ["ate", "ify", "ise", "ize"]
suffixAdjective = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
suffixAdverb = ["ward", "wards", "wise"]

In [17]:
# preprocessing datasets

def testingDataSet_preProcessing(vocabulary, file):
    preprossed = []
    
    with open(file, "r") as f:
        for index, word in enumerate(f):
            if not word.split():
                word = "--n--"
                preprossed.append(word)

            elif word.strip() not in vocabulary:
                word = tagAllotement(word)
                preprossed.append(word)

            else:
                preprossed.append(word.strip())

    return preprossed
def tagAllotement(word):
    if any(char.isdigit() for char in word):
        return "--unk_digit--"

    elif any(char in setPunctuation for char in word):
        return "--unk_punct--"

    elif any(char.isupper() for char in word):
        return "--unk_upper--"

    elif any(word.endswith(suffix) for suffix in suffixNoun):
        return "--unk_noun--"

    elif any(word.endswith(suffix) for suffix in suffixVerb):
        return "--unk_verb--"

    elif any(word.endswith(suffix) for suffix in suffixAdjective):
        return "--unk_adj--"

    elif any(word.endswith(suffix) for suffix in suffixAdverb):
        return "--unk_adv--"
    return "--unk--"

def preprocessWord(wordTagPair, vocabulary): 
    if not wordTagPair.split():
        word = "--n--"
        tag = "--s--"
        return word, tag
    else:
        word,tag = wordTagPair.split()
        if word not in vocabulary: 
            word = tagAllotement(word)
        return word, tag
    return None 

In [18]:
vocabulary = {}  

with open("WSJ_02-21.pos", 'r') as file:
    trainingDataset = file.readlines()

with open("WSJ_24.pos", 'r') as file:
    testingDataset = file.readlines()
    
with open("hmm_vocab.txt", 'r') as file:
    voc = file.read().split('\n')

i = 0
for word in voc:
    vocabulary[word] = i
    i+=1
    
#preprocess testingDataSet
preprocessedTestingData = testingDataSet_preProcessing(vocabulary, "test.words") 

In [ ]:
vocabulary

{'!': 0,
 '#': 1,
 '$': 2,
 '%': 3,
 '&': 4,
 "'": 5,
 "''": 6,
 "'40s": 7,
 "'60s": 8,
 "'70s": 9,
 "'80s": 10,
 "'86": 11,
 "'90s": 12,
 "'N": 13,
 "'S": 14,
 "'d": 15,
 "'em": 16,
 "'ll": 17,
 "'m": 18,
 "'n'": 19,
 "'re": 20,
 "'s": 21,
 "'til": 22,
 "'ve": 23,
 '(': 24,
 ')': 25,
 ',': 26,
 '-': 27,
 '--': 28,
 '--n--': 29,
 '--unk--': 30,
 '--unk_adj--': 31,
 '--unk_adv--': 32,
 '--unk_digit--': 33,
 '--unk_noun--': 34,
 '--unk_punct--': 35,
 '--unk_upper--': 36,
 '--unk_verb--': 37,
 '.': 38,
 '...': 39,
 '0.01': 40,
 '0.0108': 41,
 '0.02': 42,
 '0.03': 43,
 '0.05': 44,
 '0.1': 45,
 '0.10': 46,
 '0.12': 47,
 '0.13': 48,
 '0.15': 49,
 '0.17': 50,
 '0.19': 51,
 '0.2': 52,
 '0.24': 53,
 '0.25': 54,
 '0.3': 55,
 '0.32': 56,
 '0.375': 57,
 '0.4': 58,
 '0.43': 59,
 '0.45': 60,
 '0.5': 61,
 '0.53': 62,
 '0.59': 63,
 '0.6': 64,
 '0.60': 65,
 '0.7': 66,
 '0.71': 67,
 '0.75': 68,
 '0.8': 69,
 '0.88': 70,
 '0.9': 71,
 '0.94': 72,
 '0.95': 73,
 '1': 74,
 '1,000': 75,
 '1,012': 76,
 '1,015':

In [19]:
def calcCounts(trainingDataset, vocabulary):
    #countEmission - #(tag,word) pairs
    #countTransition - #(tag,tag) pairs
    #countTag - #tags
    countEmission = defaultdict(int)
    countTransition = defaultdict(int)
    countTag = defaultdict(int)
    
    previousTag = '--s--' 
    index = 0 
    
    for wordTagPair in trainingDataset:
        index+=1

        word, tag = preprocessWord(wordTagPair, vocabulary)
        
        countTransition[(previousTag, tag)] += 1
        countEmission[(tag, word)] += 1
        countTag[tag] += 1
        
        previousTag = tag
        
    return countEmission, countTransition, countTag

In [20]:
countEmission, countTransition, countTag = calcCounts(trainingDataset, vocabulary)

In [21]:
markovStates = sorted(countTag.keys())
len(markovStates)

46

In [22]:
def calcTransitionMatrix(countTransition, countTag, epsilon, markovStates):
    
    numberStates = len(markovStates)
    transitionMatrix = np.zeros((numberStates, numberStates))
    tagTagPair = set(countTransition.keys())

    for i in range(numberStates):
        for j in range(numberStates):
            countNextTag = 0
            key = (markovStates[i],markovStates[j])
            
            if key in countTransition: 
                countNextTag = countTransition[key]
                
            countPreviousTag = countTag[key[0]]
            transitionMatrix[i][j] = (countNextTag+epsilon)/(countPreviousTag+(numberStates*epsilon))

    return transitionMatrix
epsilon = 0.001
transitionMatrix = calcTransitionMatrix(countTransition, countTag, epsilon, markovStates)
def calcEmissionMatrix(countEmission, countTag, epsilon, markovStates, vocabulary):

    numberStates = len(markovStates)
    numberWords = len(vocabulary)
    emissionMatrix = np.zeros((numberStates, numberWords))
    tagWordPair = set(list(countEmission.keys()))
    
    for i in range(numberStates):
        for j in range(numberWords):
            countWord = 0
            key =  (markovStates[i],vocabulary[j])

            if key in countEmission: 
                countWord = countEmission[key]

            countTags = countTag[key[0]]

            emissionMatrix[i][j] = (countWord+epsilon)/(countTags+(epsilon*numberWords))

    return emissionMatrix
emissionMatrix = calcEmissionMatrix(countEmission, countTag, epsilon, markovStates,  list(vocabulary))


Viterbi Algorithm.


In [23]:
def viterbi_initialise(states, tag_counts, transitionMatrix, emissionMatrix, corpus, vocab):
    n_tags = len(tag_counts)
    best_probs = np.zeros((n_tags, len(corpus)),dtype = float)
    best_paths = np.zeros((n_tags, len(corpus)), dtype = float)
    #initialising best_probs and best_paths.

    start_index = states.index("--s--")
    for i in range(n_tags):
        if transitionMatrix[start_index, i]==0:
            best_probs[i,0] = float('-inf')
        else:
            best_probs[i,0] = transitionMatrix[start_index,i] + emissionMatrix[i,vocab[corpus[0]]]
    
    return best_probs, best_paths

In [24]:
def preprocess(vocab, data_fp):
    """
    Preprocess data
    """
    orig = []
    prep = []

    # Read data
    with open(data_fp, "r") as data_file:

        for cnt, word in enumerate(data_file):

            # End of sentence
            if not word.split():
                orig.append(word.strip())
                word = "--n--"
                prep.append(word)
                continue

            # Handle unknown words
            elif word.strip() not in vocab:
                orig.append(word.strip())
                word = tagAllotement(word)
                prep.append(word)
                continue

            else:
                orig.append(word.strip())
                prep.append(word.strip())

    assert(len(orig) == len(open(data_fp, "r").readlines()))
    assert(len(prep) == len(open(data_fp, "r").readlines()))

    return orig, prep

In [25]:
orig, t = preprocess(vocabulary, "test.words")
best_probs, best_paths = viterbi_initialise(markovStates, countTag, transitionMatrix = transitionMatrix, emissionMatrix = emissionMatrix, corpus = t, vocab = vocabulary )


In [ ]:
t

['The',
 'economy',
 "'s",
 'temperature',
 'will',
 'be',
 'taken',
 'from',
 'several',
 '--unk--',
 'points',
 'this',
 'week',
 ',',
 'with',
 'readings',
 'on',
 'trade',
 ',',
 'output',
 ',',
 'housing',
 'and',
 'inflation',
 '.',
 '--n--',
 'The',
 'most',
 'troublesome',
 'report',
 'may',
 'be',
 'the',
 'August',
 'merchandise',
 'trade',
 'deficit',
 'due',
 'out',
 'tomorrow',
 '.',
 '--n--',
 'The',
 'trade',
 'gap',
 'is',
 'expected',
 'to',
 'widen',
 'to',
 'about',
 '$',
 '9',
 'billion',
 'from',
 'July',
 "'s",
 '$',
 '7.6',
 'billion',
 ',',
 'according',
 'to',
 'a',
 'survey',
 'by',
 'MMS',
 'International',
 ',',
 'a',
 'unit',
 'of',
 'McGraw-Hill',
 'Inc.',
 ',',
 'New',
 'York',
 '.',
 '--n--',
 'Thursday',
 "'s",
 'report',
 'on',
 'the',
 'September',
 'consumer',
 'price',
 'index',
 'is',
 'expected',
 'to',
 'rise',
 ',',
 'although',
 'not',
 'as',
 'sharply',
 'as',
 'the',
 '0.9',
 '%',
 'gain',
 'reported',
 'Friday',
 'in',
 'the',
 'producer',
 

In [26]:
print(best_probs)

best_probs.shape

[[3.11627192e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.03111906e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [4.01855762e-04 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 ...
 [3.03449089e-05 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [6.25173467e-03 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]
 [7.53917235e-02 0.00000000e+00 0.00000000e+00 ... 0.00000000e+00
  0.00000000e+00 0.00000000e+00]]


(46, 34199)

In [27]:
print(len(markovStates))

46


In [ ]:
print(type(countTag))

In [ ]:
print(best_paths.shape)

In [ ]:
print(best_paths)

In [30]:
def viterbi_forward(transitionMatrix, emissionMatrix, testingDataset, best_probs, best_paths, vocab):
    n_tags = best_probs.shape[0]
    for i in range(1, len(testingDataset)):
        for j in range(n_tags):
            best_prob_i = float('-inf')
            best_path_i = None
            for k in range(n_tags):
                probability = best_probs[k,i-1]+math.log(emissionMatrix[j,vocab[testingDataset[i]]] * transitionMatrix[k,j])
                if probability > best_prob_i:
                    best_prob_i = probability
                    best_path_i = k
            best_probs[j,i]=best_prob_i
            best_paths[j,i] = best_path_i
    return best_probs, best_paths


In [31]:
a,b = viterbi_forward(transitionMatrix, emissionMatrix, t, best_probs, best_paths, vocab = vocabulary)

In [32]:
len(a)

46

In [33]:
(a)

array([[ 3.11627192e-05, -1.86286003e+01, -2.97661142e+01, ...,
        -2.08549394e+05, -2.08566684e+05, -2.08570981e+05],
       [ 7.03111906e-04, -1.75900575e+01, -3.07313922e+01, ...,
        -2.08549347e+05, -2.08561965e+05, -2.08574779e+05],
       [ 4.01855762e-04, -1.85358856e+01, -2.76636958e+01, ...,
        -2.08554904e+05, -2.08561314e+05, -2.08560049e+05],
       ...,
       [ 3.03449089e-05, -1.83233393e+01, -2.74273555e+01, ...,
        -2.08552924e+05, -2.08566830e+05, -2.08571127e+05],
       [ 6.25173467e-03, -1.89980497e+01, -2.72649196e+01, ...,
        -2.08550967e+05, -2.08559302e+05, -2.08566642e+05],
       [ 7.53917235e-02, -1.83628846e+01, -2.84602825e+01, ...,
        -2.08550928e+05, -2.08559425e+05, -2.08564942e+05]])

In [36]:

def viterbi_backward(best_probs, best_paths, corpus, states):
    corpus_words_length = len(best_paths[0]) #number of words in the corpus  = number of columns in the best_probs and best_paths matrix
    array_z = [None]* corpus_words_length
    unique_pos_tag_length = len(best_probs)  #number of unique pos tags
    last_word_best_prob = float("-inf")
    prediction_array = [None]*corpus_words_length
    for i in range(unique_pos_tag_length):
        if best_probs[i, -1]> last_word_best_prob:
            last_word_best_prob = best_probs[i, -1]
            array_z[corpus_words_length - 1] = i
            #print(i)
    prediction_array[corpus_words_length - 1] = states[array_z[corpus_words_length - 1]]
    #print(prediction_array[corpus_words_length-1], prediction_array[corpus_words_length-2])
    #print("Array_z", array_z)
    for j in range(corpus_words_length - 1, 0, -1):
        pos_tag_for_word = array_z[j]
        if(pos_tag_for_word!=None):
            pos_tag_for_word = (int)(pos_tag_for_word)
            #print(pos_tag_for_word)
        array_z[j - 1] = best_paths[int(pos_tag_for_word), j]
        prediction_array[j -1] = states[int(array_z[j - 1])]

    return prediction_array
    #return array_z


In [38]:
def viterbi_backward_test(best_probs, best_paths, corpus, states):
    corpus_words_length = len(best_paths[0]) #number of words in the corpus  = number of columns in the best_probs and best_paths matrix
    array_z = [None]* corpus_words_length
    unique_pos_tag_length = len(best_probs)  #number of unique pos tags
    last_word_best_prob = float("-inf")
    prediction_array = [None]*corpus_words_length
    for i in range(unique_pos_tag_length):
        if best_probs[i, -1]> last_word_best_prob:
            last_word_best_prob = best_probs[i, -1]
            array_z[corpus_words_length - 1] = i
            #print(i)
    prediction_array[corpus_words_length - 1] = states[array_z[corpus_words_length - 1]]
    #print(prediction_array[corpus_words_length-1], prediction_array[corpus_words_length-2])
    #print("Array_z", array_z)
    for j in range(corpus_words_length - 1, 0, -1):
        pos_tag_for_word = array_z[j]
        if(pos_tag_for_word!=None):
            pos_tag_for_word = (int)(pos_tag_for_word)
            #print(pos_tag_for_word)
        array_z[j - 1] = best_paths[int(pos_tag_for_word), j]
        prediction_array[j -1] = states[int(array_z[j - 1])]

    return array_z
    #return array_z



In [40]:
array_z = viterbi_backward_test(a, b, t, markovStates)


In [37]:
prediction_array = viterbi_backward(a, b, t, markovStates)

In [43]:
print(t[-7:])
prediction_array[-7:]

['see', 'them', 'here', 'with', 'us', '.', '--n--']


['VB', 'PRP', 'RB', 'IN', 'PRP', '.', '--s--']

In [45]:
print("third word is", t[3])
print("prediction is", prediction_array[3])
print("label is", testingDataset[3])

third word is temperature
prediction is NN
label is temperature	NN



In [46]:
#accuracy prediction
def get_accuracy(prediction_array, testingDataset):
    correct_observations = 0
    total_observations = 0

    for i,j in zip(prediction_array, testingDataset):
        dataset_word_splitted = j.split()
        testing_word_length = len(dataset_word_splitted)
        if(testing_word_length != 2):
            continue
        word = dataset_word_splitted[0]
        tag = dataset_word_splitted[1]
        if i == tag:
            correct_observations += 1
        total_observations += 1
    return correct_observations/total_observations * 100

In [47]:
print("accuracy percentage is", get_accuracy(prediction_array, testingDataset))

accuracy percentage is 95.3063647155511
